In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import sys 
sys.path.append('../../../warpspeed/')
sys.path.append('../../../measutils/')
sys.path.append('../../../pystencils_notebooks')
sys.path.append('..')

%load_ext autoreload
%autoreload 1



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from meas_db import MeasDB

from measured_metrics import MeasuredMetrics, ResultComparer
from predict_metrics import *
from plot_utils import *
import genconv
import json

In [ ]:
device = DeviceMI210()

In [ ]:
meas_db = MeasDB("bconv.db")
#meas_db.clearDB()

In [ ]:
width = 4096 
height = 4096
input_channels = 64
output_channels = 64
batch_size = 1



In [ ]:
codeString, buffers, domainSize = genconv.getCodeString( 3, width, height, input_channels, output_channels, batch_size, 1, 1,  (256, 1, 1))                        
wsKernel = genconv.getConvWarpSpeedKernel( 3, width, height, input_channels, output_channels, batch_size, 1, 1,  (256, 1, 1))

print()
print()

for f in wsKernel.loadFields:
    print(f.name)
    print(len(f.linearExpressions))

    for a in  f.linearExpressions:
        print(a)

    print()

In [ ]:
print(codeString)

In [ ]:
predValues = dict()
measValues = dict()

print(device.name)



for c_in_per_thread in [4, 8]:
    for x_per_thread in [4, 8]:
        for xblock in [16, 32, 64]:
            for yblock in [1, 2, 4, 8, 16, 32, 64, 128, 256]:
                for zblock in [1, 2]:
                    
                    if c_in_per_thread * x_per_thread >= 256:
                        continue
                    
                    
                    if x_per_thread * c_in_per_thread >= 256:
                        continue
                    if xblock * yblock * zblock != 256:
                        continue
                    if (
                        height % x_per_thread != 0
                        or input_channels % c_in_per_thread != 0
                    ):
                        continue

                    if (height % (yblock * x_per_thread) != 0
                        or input_channels % (zblock * c_in_per_thread) != 0
                    ):
                        continue

                    t1 = time.process_time()
                    blockSize = (xblock, yblock, zblock)   

                    key = (*blockSize, c_in_per_thread, x_per_thread)

                    print(key)
                
                
                    lc, basic, meas = meas_db.getEntryKeys({"input_channels": input_channels,
                                        "output_channels": output_channels,
                                        "width" : width,
                                        "height" : height,
                                        "batch_size" : batch_size,
                                        "c_in_per_thread" : c_in_per_thread,
                                        "x_per_thread" : x_per_thread,
                                        "xblock" : xblock,
                                        "yblock" : yblock,
                                        "zblock" : zblock,                
                                        "device": '"' + str(device.name) + '"'})

                    basic = None
                    meas = None
            
                    if meas is None or basic is None:
                        print("measured")
                    
                        codeString, buffers, domainSize = genconv.getCodeString( 3, width, height, input_channels, output_channels, batch_size,
                                                                                 c_in_per_thread, x_per_thread,  blockSize)
                        
                        wsKernel = genconv.getConvWarpSpeedKernel( 3, width, height, input_channels, output_channels,
                                                                   batch_size, c_in_per_thread, x_per_thread,  blockSize)
                        
                        wsKernel.registers = 20 + c_in_per_thread * x_per_thread + c_in_per_thread + x_per_thread
                        print(wsKernel.registers)
                        
                        lc = LaunchConfig.compute(wsKernel, blockSize, domainSize, (1, 1, 1), device, buffers, 0)
               
                        
                        if basic is None:
                            basic = BasicMetrics.compute(lc, device, wsKernel)
                        if meas is None:
                            meas = MeasuredMetrics.measure( codeString, lc)

                        meas_db.insertValueKeys({"input_channels": input_channels,
                                        "output_channels": output_channels,
                                        "width" : width,
                                        "height" : height,
                                        "batch_size" : batch_size,
                                        "c_in_per_thread" : c_in_per_thread,
                                        "x_per_thread" : x_per_thread,
                                        "xblock" : xblock,
                                        "yblock" : yblock,
                                        "zblock" : zblock,                
                                        "device": '"' + str(device.name) + '"'},
                                     {
                                        "basic_metrics": "'" + json.dumps(basic.__dict__) + "'",
                                        "measured_metrics": "'" + json.dumps(meas.__dict__) + "'",
                                        "launch_config": "'" + json.dumps(lc.__dict__) + "'"})
                    else:
                        print("cached")            
                                 
                                       
                    metrics = DerivedMetrics(lc, basic, device, meas)

                    measValues[key] = meas
                    predValues[key] = metrics

                    rc = ResultComparer(meas, metrics)

                    display(HTML(rc.html(referenceCount="perFlop")))

                    plotPerfProfile(metrics, meas, device)
                    
                    meas_db.commit()
                    t2 = time.process_time()
                    print("{:5.1f} ms".format((t2-t1) * 1000))
                    print()                 


In [ ]:

volumeScatterPlot([((k[0], k[1], k[2]), perFlop(measValues[k], "memLoad"), perFlop(predValues[k], "memLoadV1"), k[3]) for k in measValues], "Memory Load Volumes V1 " + str(device.name))
volumeScatterPlot([((k[0], k[1], k[2]), perFlop(measValues[k], "memLoad"), perFlop(predValues[k], "memLoadV2"), k[3], perFlop(predValues[k],"memLoadV1")) for k in measValues], "Memory Load Volumes V2 " + str(device.name))
volumeScatterPlot([((k[0], k[1], k[2]), perFlop(measValues[k], "memLoad"), perFlop(predValues[k], "memLoadV3"), k[3], perFlop(predValues[k],"memLoadV2")) for k in measValues], "Memory Load Volumes V3 " + str(device.name))
volumeScatterPlot([((k[0], k[1], k[2]), perFlop(measValues[k], "memLoad"), perFlop(predValues[k], "memLoadV4"), k[3], perFlop(predValues[k],"memLoadV3")) for k in measValues], "Memory Load Volumes V4 " + str(device.name))


In [ ]:
def fitValues(measValues, predValues):
    fig, ax = plt.subplots()
    fig.set_figwidth(5)
    fig.set_figheight(5)
    fig.set_dpi(140)

    xdata = []
    ydata = []
        


    meas = measValues
    pred = predValues
    for a in [0,1]:
        keys = [k for k in meas if (pred[k].memLoadV1 - meas[k].memLoad) > 6 and
               (pred[k].basic.waveMemLoadOverlap[a] / pred[k].basic.waveValidCells) > 6]
        x = [ max(0.0, min(40.0, device.sizeL2 / (pred[k].basic.waveMemOld[a] + pred[k].basic.waveMemLoadNew - pred[k].basic.waveMemLoadOverlap[a]))) for k in keys]
        y = [ max(0.0, min(1.0, (pred[k].memLoadV1 - meas[k].memLoad) / (pred[k].basic.waveMemLoadOverlap[a] / pred[k].basic.waveValidCells / pred[k].lc.blocking_factors[1] / pred[k].lc.blocking_factors[2])))  for k in keys]
        ax.plot(x, y, ".", alpha=0.2)
        xdata.extend(x)
        ydata.extend(y)

    values = np.arange(0.0, 10.0, 0.1)
    
    if len(keys) == 0:
        return
     
    from scipy.optimize import curve_fit

    def func(x, a, b, c):
        return a * np.exp(-b*np.exp(-c*x))


    popt, pcov = curve_fit(func, xdata, ydata, bounds=([-np.inf, -np.inf, -np.inf], [np.inf, np.inf, np.inf]), maxfev=20000)
    print(popt)
    print(pcov)

    xdata = np.array([*list(xdata), *[  i / 155 for i in range(1,155) ]])
    xdata.sort()

    
    plt.plot(xdata, func(xdata, *popt), 'r-',
             label='fit: a=%5.3f, \n     b=%5.3f,\n     c=%5.3f' % tuple(popt))

    ax.axvline(1.0)
    ax.legend()

fitValues(measValues, predValues)

In [ ]:
def fitValues(measValues, predValues):
    fig, ax = plt.subplots()
    fig.set_figwidth(5)
    fig.set_figheight(5)
    fig.set_dpi(140)

    xdata = []
    ydata = []
     

    meas = measValues
    pred = predValues
    
    keys = [k for k in meas if (meas[k].memLoad - pred[k].memLoadV2) > -10.0]
    xdata = [ max(0.0, min(420.0, pred[k].L2Oversubscription)) for k in keys]
    ydata = [ max(0.0, min(12.0, (meas[k].memLoad - pred[k].memLoadV3) / (pred[k].L2LoadV2)))  for k in keys]
      
    ax.plot(xdata, ydata, ".", alpha=0.2)
    

    values = np.arange(0.0, 10.0, 0.1)
    
    if len(keys) == 0:
        return
     
    from scipy.optimize import curve_fit

    def func(x, a, b, c):
        return a * np.exp(-b*np.exp(-c*x))


    popt, pcov = curve_fit(func, xdata, ydata, bounds=([0.1, 9.0, 0.0], [1.0, np.inf, np.inf]), maxfev=20000)
    print(popt)
    print(pcov)

    xdata = np.array([*list(xdata), *[  i / 20 for i in range(1,100) ]])
    xdata.sort()


    plt.plot(xdata, func(xdata, *popt), 'r-',
             label='fit: a=%5.3f, \n     b=%5.3f,\n     c=%5.3f' % tuple(popt))

    ax.axvline(1.0)
    ax.legend()

fitValues(measValues, predValues)

In [ ]:
volumeScatterPlot([(k[0:3], perFlop(measValues[k], "memStore"), perFlop(predValues[k], "memStoreV1"), k[3]) for k in measValues], "Memory Store Volumes V1 " + str(device.name))
volumeScatterPlot([(k[0:3], perFlop(measValues[k], "memStore"), perFlop(predValues[k], "memStoreV2"), k[3], perFlop(predValues[k], "memStoreV1"), ) for k in measValues], "Memory Store Volumes V2 " + str(device.name))


In [ ]:
volumeScatterPlot([(k[0:3], perFlop(measValues[k], "L2Load_tex"), perFlop(predValues[k], "L2LoadV1"), k[3]) for k in measValues], "Stencil L2 Load Volumes V1  " + str(device.name))
volumeScatterPlot([(k[0:3], perFlop(measValues[k], "L2Load_tex"), perFlop(predValues[k], "L2LoadV2"), k[3], perFlop(predValues[k], "L2LoadV1")) for k in measValues], "Stencil L2 Load Volumes V2 " + str(device.name))
volumeScatterPlot([(k[0:3], perFlop(measValues[k], "L2Store"), perFlop(predValues[k], "L2Store"), k[3] ) for k in measValues], "Stencil L2 Store Volumes " + str(device.name))

In [ ]:
fig, ax = volumeScatterPlot([(k[0:3], perFlop(measValues[k], "L1DataPipeWavefronts"), perFlop(predValues[k], "L1DataPipeCycles"), k[4]) for k in measValues], "L1 Data Pipe Cycles  " + str(device.name))
fig, ax = volumeScatterPlot([(k[0:3], perFlop(measValues[k], "L1TagWavefronts"), perFlop(predValues[k], "L1TagCycles") , k[4]) for k in measValues], "L1 Tage Wavefront Cycles  " + str(device.name))
fig, ax = volumeScatterPlot([(k[0:3], max( perFlop(measValues[k], "L1DataPipeWavefronts"), perFlop(measValues[k], "L1TagWavefronts")),
                                      max( perFlop(predValues[k], "L1TagCycles"), perFlop(predValues[k],"L1DataPipeCycles")), k[4]) for k in measValues], "L1 Cache Cycles  " + str(device.name))
fig, ax = volumeScatterPlot([(k[0:3], measValues[k].L1DataPipeWavefronts, measValues[k].L1TagWavefronts  , k[4]) for k in measValues], "L1 Tag vs Datapipe Cycles  " + str(device.name))
ax.set_xlabel("dataPipeWavefronts")
ax.set_ylabel("tagWavefronts")
for k in measValues:
    print("{:17s}  {:7.2f} {:7.2f} : {:7.2f} {:7.2f}".format( str(k), measValues[k].L1TagWavefronts, measValues[k].L1DataPipeWavefronts, predValues[k].L1TagCycles, predValues[k].L1DataPipeCycles))

In [ ]:
categories = ["flops", "L1", "L2", "DRAM" ]
r = 0
keys = measValues.keys()
volumeScatterPlot([(k[0:3], measValues[k].tflops, predValues[k].perfTFlopsV1, categories[predValues[k].limV1]) for k in keys], " Extended Roofline V1,  " + str(device.name))

volumeScatterPlot([(k[0:3], measValues[k].tflops, predValues[k].perfTFlopsV2, categories[predValues[k].limV2], predValues[k].perfTFlopsV1) for k in keys], "R  Extended Roofline V2,  " + str(device.name))
fig, ax = volumeScatterPlot([( k[0:3], measValues[k].tflops, predValues[k].perfTFlopsV3, categories[predValues[k].limV3], predValues[k].perfTFlopsV2) for k in keys], " Extended Roofline V3,  " + str(device.name))
ax.set_xlabel("measured performance, GFlop/s")
ax.set_ylabel("predicted performance, GFlop/s")
plt.savefig("./autoplots/extroofline_v3_range" + str(3) + "_" + device.name +  ".svg")
#volumeScatterPlot([(k[0:3], measValues[k].tflops, predValues[k].perfTFlopsV4, categories[predValues[k].limV4], predValues[k].perfTFlopsV3) for k in keys], " Extended Roofline V4,  " + str(device.name))



fig, ax = volumeScatterPlot([(k[0:3], measValues[k].tflops, predValues[k].perfTFlopsPheno, categories[predValues[k].limPheno], predValues[k].perfTFlopsV2) for k in keys], "R" + str(r) + " Extended Roofline Pheno,  " + str(device.name))
ax.set_xlabel("measured performance, GFlops/s")
ax.set_ylabel("predicted performance, GFLops/s")
plt.savefig("./autoplots/extroofline_pheno_range" + str(3) + "_" + device.name +  ".svg")

    
volumeScatterPlot([(k[0:3], measValues[k].tflops, predValues[k].perfTFlopsEPMPheno, categories[predValues[k].limPheno], predValues[k].perfTFlopsPheno) for k in keys], "R" + str(r) + " EPM Pheno,  " + str(device.name))
#volumeScatterPlot([(k[0:3], measValues[k].tflops, predValues[k].perfTFlops2LimV4, categories[predValues[k].lim2LimPheno], predValues[k].perfTFlopsPheno) for k in keys], "R" + str(r) + " Standard Roofline Pheno,  " + str(device.name))


In [ ]:
predTop = [(predValues[k].perfTFlopsV4, k[0:3], k[3], k[4]) for k in measValues.keys() ]
measTop = [(measValues[k].tflops,  k[0:3], k[3], k[4]) for k in measValues.keys() ]

print(len(predTop))

predTop = sorted(predTop)[-64:]
measTop = sorted(measTop)[-64:]


print("Top Preds:")
for p in predTop:    
    print("{} {:5.2f} {!r:12} {!r:4} {!r:4}".format("**" if p[1:] == measTop[-1][1:] else "  ", *p))

print()
print("Top Meas")
for p in measTop:    
    print("{} {:5.2f} {!r:12} {!r:4} {!r:4}".format("**" if p[1:] == predTop[-1][1:] else "  ", *p))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6), dpi=200)



balances = []



def plotLimiter(q1, q2, color):    
    machineBalance = device.peakFP32() / getattr(device, q2) *0.8
    codeBalance = [ 1 / sum([perFlop(measValues[k], m) for m in q1]) for k in measValues]
    print(machineBalance)
    
    p1 = min([measValues[k].tflops for k in measValues])  / getattr(device, q2) * 1000
    
    p2 = max(codeBalance)
    plt.plot( (  machineBalance, p2), (device.peakFP32() / 1000, device.peakFP32() / 1000), color="gray" )
    plt.plot(  (p1, machineBalance), (p1 * getattr(device, q2) / 1000, device.peakFP32() / 1000), color=color  )
    plt.scatter( codeBalance, [measValues[k].tflops for k in measValues], color=color, marker=["v", "s", "^"][plotLimiter.marker], alpha=0.6 )
    plotLimiter.marker += 1
    return codeBalance

plotLimiter.marker = 0    
    
balances.append( plotLimiter( ["memLoad", "memStore"], "memBW", "C0") )
balances.append( plotLimiter(["L2Load_tex", "L2Store"], "L2BW", "C1") )


peakRate = device.smCount * device.clock * 128
machineBalance = device.peakFP32() / peakRate

codeBalance = [1 /  perFlop(predValues[k], "L1Cycles") / 4 for k in measValues]
p1 = min([measValues[k].tflops for k in measValues]) / peakRate * 1000


print(p1)
plt.plot( (  machineBalance, 10), (device.peakFP32() / 1000, device.peakFP32() / 1000), color="gray" )
plt.plot(  (p1, machineBalance), (p1 * peakRate  / 1000, device.peakFP32() / 1000), color="C2"  )
plt.scatter( codeBalance, [measValues[k].tflops for k in measValues], color="C2", alpha=0.6 )

balances.append( codeBalance )


for k in range(len(balances[0])):
    plt.plot( (balances[0][k], balances[1][k]), [ measValues[ list(measValues.keys())[k] ].tflops ] * 2, color= getColor(list(measValues.keys())[k][0:3]), zorder=-1, alpha=0.2  )
    plt.plot( (balances[1][k], balances[2][k]), [ measValues[ list(measValues.keys())[k] ].tflops ] * 2, color=getColor(list(measValues.keys())[k][0:3]), zorder=-1, alpha=0.2  )

ax.set_xscale("log")
ax.set_yscale("log")

ax.set_xlabel("arithmetic Intensity, Flop/B or Flop/cycle")
ax.set_ylabel("Performance, GFlop/s")


In [ ]:
from math import log, exp

def L2Latency(wdev, payload, clock):

    # return max(200, (payload * wdev) / 1500 * clock)
    x = wdev * payload / 8 / 32
    # lat = 210 + 102* log(1 + exp((x-1400)*0.00138))
    lat = max(237, 175 + 440 * log(1 + exp((x - 1000) * 0.0002)))
    # print("{:6.0f} {:6.0f} {:6.1f}".format(x, lat, payload / 8 / 32))
    return lat


def memLatency(wdev, payload, clock):
    # return max(250, wdev*payload / 780 * clock)
    x = wdev * payload / 8 / 32
    lat = max(210, 0 + 1300 * log(1 + exp((x - 1000) * 0.00017)))
    # print("{:6.0f} {:6.0f} {:6.1f}".format(x, lat, payload / 8 / 32))
    return lat


for x in range(1, 108):
    print(memLatency(x * 2048, 100, 1.41))
    print( max(210, x * 2048 * 100 / 1600e9 * 1.41e9))
    print()


In [ ]:
from random import *
from math import sqrt

nclusters = min(4, len(measValues.keys() ) // 2)

keys = list(measValues.keys())
shuffle(keys)


clusters = [[] for n in range(nclusters)]
clusterCenters = [keys[i] for i in range(nclusters)]


fakeValues = {}
for key  in measValues.keys():
    fakeValues[key] = [randint(0, 20), randint(0, 20)]  


def distance(key1, key2):
    #return ((fakeValues[key1][0] - fakeValues[key2][0])**2 + (fakeValues[key1][1] - fakeValues[key2][1])**2)
    return sqrt(( (perFlop(measValues[key1], "memLoad") - perFlop(measValues[key2], "memLoad")) / device.memBW  )**2 +
                ( (perFlop(measValues[key1], "L2Load_tex") - perFlop(measValues[key2], "L2Load_tex")) / device.L2BW ) **2) 
    

for i in range(0,10):

    for c in range(nclusters):
        print(len(clusters[c]))
        if len(clusters[c]) <= 1 or randint(0,2000) == 0:
            clusterCenters[c] = choice(keys)

    clusters = [[] for n in range(nclusters)]    
    for key in keys:
        minDistance = -1
        minCluster = -1
        for c in range(nclusters):            
            d = distance(key, clusterCenters[c])

            if d < minDistance or minDistance < 0:
                minDistance = d
                minCluster = c
        clusters[minCluster].append(key)
            
    for c in range(nclusters):
        shuffle(clusters[c])
        lowestDistanceSum = -1
        lowestKey = -1
        for k1 in clusters[c]:
            distanceSum = 0
            for k2 in clusters[c]:
                distanceSum += distance(k1, k2)
            if distanceSum < lowestDistanceSum or lowestDistanceSum < 0:
                lowestDistanceSum = distanceSum
                lowestKey = k1
        if c != -1:
            clusterCenters[c] = lowestKey
        
            
  
        
    print(clusterCenters)
          
    keyClusters = {}
    for k in keys:
        for c in range(nclusters):
            if k in clusters[c]:
                keyClusters[k] = c

    fig, ax = plt.subplots(figsize=[5, 5], dpi=200)
    #volumeScatterPlot([(k[0:3], perFlop(measValues[k], "memLoad"), perFlop(measValues[k], "L2Load_tex"), keyClusters[k]) for k in keys], "test_" + str(device.name))    
    for c in range(nclusters):
        ax.scatter( [perFlop(measValues[k], "memLoad") / device.memBW for k in clusters[c]], [perFlop(measValues[k], "L2Load_tex") / device.L2BW for k in clusters[c]])
        if clusterCenters[c] != -1:
            ax.plot( perFlop(measValues[clusterCenters[c]], "memLoad") / device.memBW,  perFlop(measValues[clusterCenters[c]], "L2Load_tex") / device.L2BW, "+", markersize=15, color="black" )
        #ax.scatter( [fakeValues[k][0] for k in clusters[c]], [fakeValues[k][1] for k in clusters[c]])        

    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    ax.set_xlim(min(xlim[0], ylim[0]), max(xlim[1], ylim[1])  )
    ax.set_ylim(min(xlim[0], ylim[0]), max(xlim[1], ylim[1])  )
        
    plt.show()
    plt.close(fig)



